In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

from langchain_community.chat_models import ChatZhipuAI
from llama_index.core import Settings
zhipuai_chat = ChatZhipuAI(
    temperature=0.5,
    api_key="8da4437242119e94b6e5b8d6ba4b420d.NSaFt61REgNGu1kS",
    model="glm-4-flash",
    # api_base="...",
    # other params...
)
Settings.llm = zhipuai_chat

In [2]:
from llama_index.embeddings.openai import OpenAIEmbedding
# from langchain_community.embeddings import ZhipuAIEmbeddings
# from llama_index.embeddings.langchain import LangchainEmbedding

# Settings.embed_model = LangchainEmbedding(ZhipuAIEmbeddings(
#     model="embedding-3",
#     api_key="8da4437242119e94b6e5b8d6ba4b420d.NSaFt61REgNGu1kS",
#     dimensions=1024  # Specify the size of the embeddings to be returned
# ))
embedding_model = "text-embedding-3-small"
Settings.embed_model = OpenAIEmbedding(api_key='sk-bmzsXFgX4CgCUCoC22IgfGhRomJLvd2M8EI74JuNWij3YRPJ',
                                       api_base="https://api.agicto.cn/v1",
                                       model_name=embedding_model)  # Changed 'model' to 'model_name'


In [3]:
import os
import pickle
from pathlib import Path
import nest_asyncio
import nltk
from llama_index.core import VectorStoreIndex, Settings
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import UnstructuredElementNodeParser

# Allow nested event loops
nest_asyncio.apply()

# Load all files in data

file_path = Path("./data")
data = []
for file in file_path.glob('*'):  # Iterate through all files in the directory
    data.extend(FlatReader().load_data(file))  # Load data from each file and extend the list

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

# Parse nodes
node_parser = UnstructuredElementNodeParser()
raw_nodes = []  # Initialize raw_nodes to avoid NameError
if not os.path.exists("qr_2023_nodes.pkl"):
    raw_nodes = node_parser.get_nodes_from_documents(data,show_progress=True)
    pickle.dump(raw_nodes, open("qr_2023_nodes.pkl", "wb"))
else:
    with open("qr_2023_nodes.pkl", "rb") as f:
        raw_nodes = pickle.load(f)  # Load existing raw_nodes if the file exists

base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(raw_nodes)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jiancao/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jiancao/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/jiancao/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
0it [00:00, ?it/s]%|          | 0/1 [00:00<?, ?it/s]
Parsing nodes: 100%|██████████| 1/1 [00:04<00:00,  4.52s/it]


In [4]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# Create vector index and retriever
vector_index = VectorStoreIndex(base_nodes,show_progress=True)
retriever = vector_index.as_retriever(similarity_top_k=3)

# Set up recursive retriever and query engine
recursive_retriever = RecursiveRetriever("vector", {"vector": retriever}, node_mappings)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

Generating embeddings:   0%|          | 0/133 [00:00<?, ?it/s]

APIConnectionError: Connection error.

In [ ]:
response = query_engine.query("Who is the top 10 richest billionaire in 2015? List his name, new worth, age, nationality and source of wealth")
print(response)

In [ ]:
response = query_engine.query("Growth of total gross margin of tesla in 2020?")
print(response)